In [264]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle



In [265]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [266]:
#preprocess data
data = data.drop(['RowNumber'],axis=1)
data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [267]:
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,Obijiaku,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,15569892,Johnstone,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,15584532,Liu,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,15682355,Sabbatini,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [268]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()
geo_onehotencoder = onehot_encoder.fit_transform(data[['Geography']])
geo_onehotencoder


<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [269]:
onehot_encoder.get_feature_names_out(['Geography'])
geo_onehotencoder.toarray()


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [270]:
geo_encoded_df = pd.DataFrame(geo_onehotencoder.toarray(),columns =onehot_encoder.get_feature_names_out(['Geography']) )
geo_encoded_df
data= pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,15634602,Hargrave,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,15647311,Hill,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,15619304,Onio,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,15701354,Boni,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,15737888,Mitchell,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [271]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('geo_onehotencoder.pkl','wb') as file:
    pickle.dump(geo_onehotencoder,file)   

In [272]:
data = data.drop('Surname',axis=1)
X=data.drop('Exited',axis=1)
y=data['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)



In [273]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping ,TensorBoard
import datetime

In [274]:

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Correct input shape
    Dense(32, activation='relu'),  # Add comma here
    Dense(1, activation='sigmoid')  # Add comma before closing ]
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
'''model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")'''
model.summary()



/Users/armangarg/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_30 (Dense)                │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

In [275]:
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = "logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [276]:
early_stopping = EarlyStopping(monitor = 'val_loss',patience=15,restore_best_weights=True)

In [277]:
#train the model
history=model.fit(
    
    X_train,y_train,validation_data = (X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping]
    
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8022 - loss: 0.4913 - val_accuracy: 0.8310 - val_loss: 0.4043
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - accuracy: 0.8275 - loss: 0.4109 - val_accuracy: 0.8515 - val_loss: 0.3644
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - accuracy: 0.8493 - loss: 0.3574 - val_accuracy: 0.8575 - val_loss: 0.3456
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - accuracy: 0.8597 - loss: 0.3454 - val_accuracy: 0.8625 - val_loss: 0.3412
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - accuracy: 0.8607 - loss: 0.3390 - val_accuracy: 0.8620 - val_loss: 0.3410
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - accuracy: 0.8629 - loss: 0.3236 - val_accuracy: 0.8565 - val_loss: 0.3430
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.8690 - loss: 0.3261 - val_accuracy: 0.8630 - val_loss: 0.3392
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - accuracy: 0.8687 - loss: 0.3

In [278]:
model.save('model.h5')